# Spider 데이터셋을 사용한 Llama 3.1 파인튜닝 (테이블 스키마 통합)

In [3]:
import os
os.environ["PYTORCH_TRITON"] = "0"
os.environ['TRITON_JIT_DISABLE_OPT'] = '1'

In [4]:
# Google Drive 연결
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정 (Spider 데이터셋이 저장된 경로로 수정하세요)
SPIDER_DIR = '/content/drive/MyDrive/spider/spider_data'  # 필요에 따라 수정
DATASET_PATH = '/content/drive/MyDrive/spider/spider_data/spider_train_translations_final.csv'  # 필요에 따라 수정

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:

!pip uninstall unsloth triton -y
!pip install triton vllm unsloth tqdm pandas

Found existing installation: unsloth 2025.5.7
Uninstalling unsloth-2025.5.7:
  Successfully uninstalled unsloth-2025.5.7
Found existing installation: triton 3.2.0
Uninstalling triton-3.2.0:
  Successfully uninstalled triton-3.2.0
  Using cached triton-3.3.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.5 kB)
  Using cached unsloth-2025.5.7-py3-none-any.whl.metadata (47 kB)
  Using cached triton-3.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
Using cached triton-3.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (253.2 MB)
Using cached unsloth-2025.5.7-py3-none-any.whl (265 kB)


In [5]:
# 스키마 정보 추출 및 데이터셋 준비 코드
import os
import json
import pandas as pd
from tqdm import tqdm

def load_tables_data(spider_dir):
    """tables.json 파일에서 테이블 데이터 로드."""
    tables_path = os.path.join(spider_dir, "tables.json")

    if not os.path.exists(tables_path):
        print(f"Error: {tables_path} not found!")
        # 다른 가능한 경로 시도
        alt_paths = [
            os.path.join(spider_dir, "evaluation_examples/examples/tables.json"),
            os.path.join(spider_dir, "examples/tables.json")
        ]

        for alt_path in alt_paths:
            if os.path.exists(alt_path):
                print(f"Found tables.json at alternative path: {alt_path}")
                tables_path = alt_path
                break
        else:
            print("Please check your Spider dataset path or provide the correct path to tables.json")
            return None

    print(f"Loading tables from: {tables_path}")
    with open(tables_path, 'r') as f:
        return json.load(f)


def create_schema_mapping(tables_data):
    """db_id에서 스키마 정보로의 매핑 생성."""
    schema_mapping = {}

    for item in tables_data:
        db_id = item["db_id"]
        tables = item["table_names_original"]
        columns = {}
        column_types = {}

        # 열 정보 처리
        for i, (table_id, col_name) in enumerate(item["column_names_original"]):
            if table_id == -1:
                # 특수 "*" 열 건너뛰기
                continue

            table_name = tables[table_id]

            if table_name not in columns:
                columns[table_name] = []
                column_types[table_name] = {}

            # 가능한 경우 열 유형 추가
            col_type = ""
            if "column_types" in item and i < len(item["column_types"]):
                col_type = item["column_types"][i]
                column_types[table_name][col_name] = col_type

            columns[table_name].append(col_name)

        # 외래 키 매핑
        foreign_keys = []
        for fk_pair in item.get("foreign_keys", []):
            if len(fk_pair) == 2:
                col1_idx, col2_idx = fk_pair

                if (col1_idx < len(item["column_names_original"]) and
                    col2_idx < len(item["column_names_original"])):

                    table1_idx, col1_name = item["column_names_original"][col1_idx]
                    table2_idx, col2_name = item["column_names_original"][col2_idx]

                    if table1_idx >= 0 and table2_idx >= 0:
                        table1_name = tables[table1_idx]
                        table2_name = tables[table2_idx]
                        foreign_keys.append({
                            "source": f"{table1_name}.{col1_name}",
                            "target": f"{table2_name}.{col2_name}"
                        })

        # 기본 키 매핑
        primary_keys = []
        for pk in item.get("primary_keys", []):
            if pk < len(item["column_names_original"]):
                table_id, col_name = item["column_names_original"][pk]
                if table_id >= 0:
                    table_name = tables[table_id]
                    primary_keys.append(f"{table_name}.{col_name}")

        schema_mapping[db_id] = {
            "tables": tables,
            "columns": columns,
            "column_types": column_types,
            "foreign_keys": foreign_keys,
            "primary_keys": primary_keys
        }

    return schema_mapping


def get_schema_text(db_id, schema_mapping):
    """주어진 db_id에 대한 스키마 텍스트 생성."""
    if db_id not in schema_mapping:
        return "Schema information not available"

    schema = schema_mapping[db_id]
    tables = schema["tables"]
    columns = schema["columns"]
    column_types = schema["column_types"]
    foreign_keys = schema["foreign_keys"]
    primary_keys = schema["primary_keys"]

    schema_text = "데이터베이스 스키마:\n"

    for table in tables:
        schema_text += f"테이블: {table}\n"
        if table in columns:
            schema_text += "컬럼:\n"
            for column in columns[table]:
                # 기본 키 표시기 추가
                pk_marker = " (기본 키)" if f"{table}.{column}" in primary_keys else ""

                # 가능한 경우 열 유형 추가
                type_info = ""
                if table in column_types and column in column_types[table]:
                    type_info = f" ({column_types[table][column]})"

                schema_text += f"  - {column}{type_info}{pk_marker}\n"

    if foreign_keys:
        schema_text += "\n외래 키 관계:\n"
        for fk in foreign_keys:
            schema_text += f"  - {fk['source']} → {fk['target']}\n"

    return schema_text


def add_schema_to_dataset(csv_path, output_path, schema_mapping):
    """데이터셋의 각 예제에 스키마 정보 추가."""
    # CSV 파일 로드
    df = pd.read_csv(csv_path)

    if 'db_id' not in df.columns:
        print("Error: 'db_id' column not found in the CSV file!")
        return None

    # 스키마 정보를 위한 새 열 생성
    df['schema_info'] = ""

    # 각 행에 스키마 정보 추가
    print("Adding schema information to dataset...")
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        db_id = row['db_id']
        df.at[idx, 'schema_info'] = get_schema_text(db_id, schema_mapping)

    # 향상된 데이터셋 저장
    df.to_csv(output_path, index=False)
    print(f"Enhanced dataset saved to {output_path}")
    return df

# 테이블 데이터 로드 및 스키마 매핑 생성
print(f"Loading tables data from {SPIDER_DIR}...")
tables_data = load_tables_data(SPIDER_DIR)

if tables_data:
    print("Creating schema mapping...")
    schema_mapping = create_schema_mapping(tables_data)
    print(f"Schema mapping created for {len(schema_mapping)} databases")

    # 스키마 정보가 추가된 데이터셋 생성
    output_csv = '/content/spider_train_with_schema.csv'
    df_with_schema = add_schema_to_dataset(DATASET_PATH, output_csv, schema_mapping)

    # 결과 확인
    if df_with_schema is not None:
        print("\nSample row with schema information:")
        print(df_with_schema.iloc[0][['question_ko', 'schema_info', 'query']].to_dict())
else:
    print("Failed to load tables data. Please check the path to tables.json")

Loading tables data from /content/drive/MyDrive/spider/spider_data...
Loading tables from: /content/drive/MyDrive/spider/spider_data/tables.json
Creating schema mapping...
Schema mapping created for 166 databases
Adding schema information to dataset...


100%|██████████| 7000/7000 [00:00<00:00, 11172.49it/s]


Enhanced dataset saved to /content/spider_train_with_schema.csv

Sample row with schema information:
{'question_ko': '부서장들 중 56세보다 나이가 많은 사람이 몇 명입니까?', 'schema_info': '데이터베이스 스키마:\n테이블: department\n컬럼:\n  - Department_ID (number) (기본 키)\n  - Name (text)\n  - Creation (text)\n  - Ranking (number)\n  - Budget_in_Billions (number)\n  - Num_Employees (number)\n테이블: head\n컬럼:\n  - head_ID (number) (기본 키)\n  - name (text)\n  - born_state (text)\n  - age (number)\n테이블: management\n컬럼:\n  - department_ID (number) (기본 키)\n  - head_ID (number)\n  - temporary_acting (text)\n\n외래 키 관계:\n  - management.head_ID → head.head_ID\n  - management.department_ID → department.Department_ID\n', 'query': 'SELECT count(*) FROM head WHERE age > 56'}


## 3. 모델 및 토크나이저 로드

In [9]:
from unsloth import FastLanguageModel
import torch

# 모델 구성
max_seq_length = 2048  # 필요에 따라 조정
dtype = None  # 자동 감지
load_in_4bit = True
model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"

# 모델 및 토크나이저 로드
print("Loading model and tokenizer...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# LoRA 파인튜닝 설정
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-22 03:45:09 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-22 03:45:09 [__init__.py:239] Automatically detected platform cuda.
Loading model and tokenizer...
==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2025.5.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### 저장된 모델을 불러오는 코드

In [6]:
from unsloth import FastLanguageModel
import torch

output_dir = "/content/drive/MyDrive/spider/spider_schema_model"

# 모델과 토크나이저 로드
model, tokenizer = FastLanguageModel.from_pretrained(
    output_dir,  # 저장한 디렉토리
    device_map="auto",
    load_in_4bit=True  # 양자화 사용 시
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-23 00:59:17 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-23 00:59:17 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Unsloth 2025.5.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## 4. 데이터셋 준비

In [7]:
# 준비된 테이블 스키마 데이터셋 로드
from datasets import load_dataset
dataset = load_dataset("csv", data_files="/content/spider_train_with_schema.csv")

def formatting_prompts_with_schema(examples):
    instructions = examples["question_ko"]
    outputs = examples["query"]
    schemas = examples["schema_info"]

    formatted_chats = []

    for instruction, output, schema in zip(instructions, outputs, schemas):
        # 입력에 스키마 정보를 포함한 프롬프트 구성
        input_with_schema = f"당신은 자연어 질문을 SQL 쿼리로 변환하는 전문 AI 어시스턴트입니다. 사용자가 데이터베이스에서 정보를 얻기 위해 일상 언어로 질문하면, 당신은 해당 질문을 정확한 SQL 쿼리로 변환해야 합니다.\n\n{schema}\n\n질문: {instruction}"

        # 채팅 형식으로 변환
        chat = [
            {"role": "user", "content": input_with_schema},
            {"role": "assistant", "content": output}
        ]

        # apply_chat_template 적용
        formatted_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False)
        formatted_chats.append(formatted_text)

    return {"text": formatted_chats}

# 데이터셋 처리
processed_dataset = dataset.map(formatting_prompts_with_schema, batched=True)

# 불필요한 컬럼 제거
columns_to_remove = ["db_id", "question", "query_toks", "query_toks_no_value",
                     "question_toks", "query", "question_ko", "schema_info"]
train_dataset = processed_dataset.map(remove_columns=columns_to_remove, batched=True)["train"]

# 결과 확인
print("Dataset processed. Sample example:")
print(train_dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Dataset processed. Sample example:
{'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n당신은 자연어 질문을 SQL 쿼리로 변환하는 전문 AI 어시스턴트입니다. 사용자가 데이터베이스에서 정보를 얻기 위해 일상 언어로 질문하면, 당신은 해당 질문을 정확한 SQL 쿼리로 변환해야 합니다.\n\n데이터베이스 스키마:\n테이블: department\n컬럼:\n  - Department_ID (number) (기본 키)\n  - Name (text)\n  - Creation (text)\n  - Ranking (number)\n  - Budget_in_Billions (number)\n  - Num_Employees (number)\n테이블: head\n컬럼:\n  - head_ID (number) (기본 키)\n  - name (text)\n  - born_state (text)\n  - age (number)\n테이블: management\n컬럼:\n  - department_ID (number) (기본 키)\n  - head_ID (number)\n  - temporary_acting (text)\n\n외래 키 관계:\n  - management.head_ID → head.head_ID\n  - management.department_ID → department.Department_ID\n\n\n질문: 부서장들 중 56세보다 나이가 많은 사람이 몇 명입니까?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSELECT count(*) FROM head WHERE age > 56<|eot_id|>

## 5. 트레이너 설정 및 파인튜닝 시작

In [11]:
# GPU 메모리 상태 체크
if torch.cuda.is_available():
    gpu_stats = torch.cuda.get_device_properties(0)
    start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
    print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
    print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.137 GB of memory reserved.


In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# 트레이너 설정
trainer = SFTTrainer(
      model=model,
      tokenizer=tokenizer,
      train_dataset=train_dataset,
      dataset_text_field="text",
      max_seq_length=2048,  # 적당한 길이
      dataset_num_proc=2,   # 병렬 처리
      args=TrainingArguments(
          per_device_train_batch_size=2,
          gradient_accumulation_steps=4,
          learning_rate=5e-4,  # 학습률 증가
          fp16=True,
          logging_steps=5,
          optim="adamw_8bit",  # 8비트 옵티마이저
          num_train_epochs = 1,
          weight_decay=0.01,
          lr_scheduler_type="cosine",  # 코사인 스케줄러
          warmup_ratio=0.05,  # 웜업 적용
          seed=42,
          output_dir="outputs",
          report_to="none",
          gradient_checkpointing=True,
          eval_strategy="no",  # 평가 단계 생략
          save_strategy="steps",
          save_steps=100,   # 100 스텝마다 저장
      ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/7000 [00:00<?, ? examples/s]

In [13]:
# 파인튜닝 시작
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,000 | Num Epochs = 1 | Total steps = 875
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,1.225200
10,0.974400
15,0.547000
20,0.477500
25,0.431200
30,0.339800
35,0.374400
40,0.304200
45,0.299500
50,0.261700


Step,Training Loss
5,1.225200
10,0.974400
15,0.547000
20,0.477500
25,0.431200
30,0.339800
35,0.374400
40,0.304200
45,0.299500
50,0.261700


## 6. 결과 및 모델 저장

In [17]:
# GPU 메모리 및 학습 시간 통계
if torch.cuda.is_available():
    used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
    used_percentage = round(used_memory / max_memory * 100, 3)
    lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
    print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
    print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
    print(f"Peak reserved memory = {used_memory} GB.")
    print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
    print(f"Peak reserved memory % of max memory = {used_percentage} %.")
    print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

14350.9361 seconds used for training.
239.18 minutes used for training.
Peak reserved memory = 7.584 GB.
Peak reserved memory for training = 0.447 GB.
Peak reserved memory % of max memory = 51.448 %.
Peak reserved memory for training % of max memory = 3.032 %.


In [18]:
# 모델 저장
output_dir = "/content/drive/MyDrive/spider/spider_schema_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model saved to {output_dir}")

Model saved to /content/drive/MyDrive/spider/spider_schema_model


## 7. 추론 테스트

In [8]:
# 추론을 위한 설정
FastLanguageModel.for_inference(model)

def generate_sql_query_with_schema(question, db_id):
    # 스키마 정보 가져오기
    if db_id in schema_mapping:
        schema_text = get_schema_text(db_id, schema_mapping)
    else:
        schema_text = "Schema information not available"

    # 스키마를 포함한 입력 구성
    input_with_schema = f"당신은 자연어 질문을 SQL 쿼리로 변환하는 전문 AI 어시스턴트입니다. 사용자가 데이터베이스에서 정보를 얻기 위해 일상 언어로 질문하면, 당신은 해당 질문을 정확한 SQL 쿼리로 변환해야 합니다.\n\n{schema_text}\n\n질문: {question}"

    # 채팅 메시지로 포맷팅
    messages = [{"role": "user", "content": input_with_schema}]

    # 채팅 템플릿 적용
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # SQL 쿼리 생성
    outputs = model.generate(
        inputs,
        max_new_tokens=150,
        temperature=0.1,
        do_sample=True,
        use_cache=True
    )

    # 응답 디코딩
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 생성된 SQL 쿼리만 추출
    assistant_part = response.split(input_with_schema)[-1].strip()

    return assistant_part

In [9]:
# 예시로 테스트
test_dbs = ["department_management"]  # 처음 3개의 데이터베이스 사용
test_questions = [
    "한 번에 3을 초과하는 부서장 있는 부서의 ID, 이름과 부서장 수를 알려주쇼"
]

print("\nTesting with examples:")
for i, (question, db_id) in enumerate(zip(test_questions, test_dbs)):
    print(f"\nExample {i+1}:")
    print(f"Question: {question}")
    print(f"Database: {db_id}")

    sql = generate_sql_query_with_schema(question, db_id)
    print(f"Generated SQL: {sql}")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Testing with examples:

Example 1:
Question: 한 번에 3을 초과하는 부서장 있는 부서의 ID, 이름과 부서장 수를 알려주쇼
Database: department_management
Generated SQL: assistant

SELECT T1.department_id, T1.name, count(*) FROM department AS T1 JOIN management AS T2 ON T1.department_id = T2.department_id GROUP BY T1.department_id HAVING count(*) > 3


## 8. TextStreamer를 사용한 연속 추론 (선택 사항)

In [ ]:
from transformers import TextStreamer

def generate_sql_with_streamer(question, db_id):
    # 스키마 정보 가져오기
    if db_id in schema_mapping:
        schema_text = get_schema_text(db_id, schema_mapping)
    else:
        schema_text = "Schema information not available"

    # 스키마를 포함한 입력 구성
    input_with_schema = f"당신은 자연어 질문을 SQL 쿼리로 변환하는 전문 AI 어시스턴트입니다. 사용자가 데이터베이스에서 정보를 얻기 위해 일상 언어로 질문하면, 당신은 해당 질문을 정확한 SQL 쿼리로 변환해야 합니다.\n\n{schema_text}\n\n질문: {question}"

    # 채팅 메시지로 포맷팅
    messages = [{"role": "user", "content": input_with_schema}]

    # 채팅 템플릿 적용
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # TextStreamer 설정
    text_streamer = TextStreamer(tokenizer)

    print("\nGenerating SQL with TextStreamer:")
    # SQL 쿼리 생성 및 스트리밍
    _ = model.generate(
        inputs,
        streamer=text_streamer,
        max_new_tokens=150,
        temperature=0.1,
        do_sample=True,
        use_cache=True
    )

# 예제 시연
question = "영어 과목을 수강하는 모든 학생의 이름과 학년을 알려주세요."
db_id = test_dbs[0]  # 첫 번째 테스트 데이터베이스 사용

print(f"Question: {question}")
print(f"Database: {db_id}")
generate_sql_with_streamer(question, db_id)

# 9. 테스트 수행


In [37]:
## 10. 평가 (Evaluation)

# 필요한 라이브러리 가져오기
import json
import os
import sqlite3
import sys
import random
from tqdm import tqdm

# 평가 도우미 함수
def load_tables_data(tables_file):
    """테이블 스키마 정보를 로드합니다."""
    if not os.path.exists(tables_file):
        print(f"오류: {tables_file}를 찾을 수 없습니다!")
        return None

    print(f"{tables_file}에서 테이블 로딩 중...")
    with open(tables_file, 'r') as f:
        return json.load(f)

def create_schema_mapping(tables_data):
    """db_id에서 스키마 정보로의 매핑 생성."""
    schema_mapping = {}

    for item in tables_data:
        db_id = item["db_id"]
        tables = item["table_names_original"]
        columns = {}
        column_types = {}

        # 열 정보 처리
        for i, (table_id, col_name) in enumerate(item["column_names_original"]):
            if table_id == -1:
                # 특수 "*" 열 건너뛰기
                continue

            table_name = tables[table_id]

            if table_name not in columns:
                columns[table_name] = []
                column_types[table_name] = {}

            # 가능한 경우 열 유형 추가
            col_type = ""
            if "column_types" in item and i < len(item["column_types"]):
                col_type = item["column_types"][i]
                column_types[table_name][col_name] = col_type

            columns[table_name].append(col_name)

        # 외래 키 매핑
        foreign_keys = []
        for fk_pair in item.get("foreign_keys", []):
            if len(fk_pair) == 2:
                col1_idx, col2_idx = fk_pair

                if (col1_idx < len(item["column_names_original"]) and
                    col2_idx < len(item["column_names_original"])):

                    table1_idx, col1_name = item["column_names_original"][col1_idx]
                    table2_idx, col2_name = item["column_names_original"][col2_idx]

                    if table1_idx >= 0 and table2_idx >= 0:
                        table1_name = tables[table1_idx]
                        table2_name = tables[table2_idx]
                        foreign_keys.append({
                            "source": f"{table1_name}.{col1_name}",
                            "target": f"{table2_name}.{col2_name}"
                        })

        # 기본 키 매핑
        primary_keys = []
        for pk in item.get("primary_keys", []):
            if pk < len(item["column_names_original"]):
                table_id, col_name = item["column_names_original"][pk]
                if table_id >= 0:
                    table_name = tables[table_id]
                    primary_keys.append(f"{table_name}.{col_name}")

        schema_mapping[db_id] = {
            "tables": tables,
            "columns": columns,
            "column_types": column_types,
            "foreign_keys": foreign_keys,
            "primary_keys": primary_keys
        }

    return schema_mapping

def get_schema_text(db_id, schema_mapping):
    """주어진 db_id에 대한 스키마 텍스트 생성."""
    if db_id not in schema_mapping:
        return "Schema information not available"

    schema = schema_mapping[db_id]
    tables = schema["tables"]
    columns = schema["columns"]
    column_types = schema["column_types"]
    foreign_keys = schema["foreign_keys"]
    primary_keys = schema["primary_keys"]

    schema_text = "데이터베이스 스키마:\n"

    for table in tables:
        schema_text += f"테이블: {table}\n"
        if table in columns:
            schema_text += "컬럼:\n"
            for column in columns[table]:
                # 기본 키 표시기 추가
                pk_marker = " (기본 키)" if f"{table}.{column}" in primary_keys else ""

                # 가능한 경우 열 유형 추가
                type_info = ""
                if table in column_types and column in column_types[table]:
                    type_info = f" ({column_types[table][column]})"

                schema_text += f"  - {column}{type_info}{pk_marker}\n"

    if foreign_keys:
        schema_text += "\n외래 키 관계:\n"
        for fk in foreign_keys:
            schema_text += f"  - {fk['source']} → {fk['target']}\n"

    return schema_text

def generate_sql_query(model, tokenizer, schema_mapping, question, db_id):
    """주어진 질문과 데이터베이스에 대한 SQL 쿼리를 생성합니다."""
    try:
        # 스키마 텍스트 가져오기
        if db_id in schema_mapping:
            schema_text = get_schema_text(db_id, schema_mapping)
        else:
            schema_text = "Schema information not available"

        # 스키마를 포함한 입력 구성
        input_with_schema = f"당신은 자연어 질문을 SQL 쿼리로 변환하는 전문 AI 어시스턴트입니다. 사용자가 데이터베이스에서 정보를 얻기 위해 일상 언어로 질문하면, 당신은 해당 질문을 정확한 SQL 쿼리로 변환해야 합니다.\n\n{schema_text}\n\n질문: {question}"

        # 채팅 메시지로 포맷팅
        messages = [{"role": "user", "content": input_with_schema}]

        # 채팅 템플릿 적용
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        # SQL 쿼리 생성
        outputs = model.generate(
            inputs,
            max_new_tokens=150,
            temperature=0.1,
            do_sample=True,
            use_cache=True
        )

        # 응답 디코딩
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # 생성된 SQL 쿼리 추출
        assistant_part = response.split(input_with_schema)[-1].strip()

        # 응답 정리하여 SQL 쿼리만 추출
        if assistant_part.startswith("assistant"):
            assistant_part = assistant_part[len("assistant"):].strip()

        # SQL과 유사한 첫 번째 문장 찾기
        lines = assistant_part.split("\n")
        sql_query = ""
        in_sql = False
        for line in lines:
            line = line.strip()
            if not line:
                continue

            # SQL 키워드로 쿼리 시작 식별
            if not in_sql and (line.lower().startswith("select") or
                             line.lower().startswith("with") or
                             line.lower().startswith("(select")):
                in_sql = True

            if in_sql:
                sql_query += line + " "

                # 코드 펜스나 종료 표시를 찾으면 중지
                if line.endswith(";") or line.endswith("```"):
                    break

        # 명확한 SQL 쿼리를 찾지 못한 경우 전체 응답 사용
        if not sql_query:
            sql_query = assistant_part

        # 코드 마커, 백틱 등 정리
        sql_query = sql_query.replace("```sql", "").replace("```", "").strip()

        return sql_query

    except Exception as e:
        print(f"SQL 쿼리 생성 오류: {str(e)}")
        return "SELECT * FROM table"  # 오류 발생 시 기본 쿼리

def evaluate_predictions(preds, golds, db_dir, tables_file):
    """
    생성된 SQL 예측을 평가합니다.

    Args:
        preds: [(sql, db_id), ...] 형식의 예측 목록
        golds: [(sql, db_id), ...] 형식의 정답 목록
        db_dir: SQLite 데이터베이스가 있는 디렉토리
        tables_file: tables.json 파일 경로

    Returns:
        정확도 점수를 포함하는 딕셔너리
    """
    # Spider evaluation.py와 유사한 평가 로직 구현
    scores = {
        'exact_match': 0,
        'execution': 0,
        'total': len(preds),
        'details': []  # 각 예제별 세부 평가 결과
    }

    for i, ((pred_sql, db_id), (gold_sql, _)) in enumerate(zip(preds, golds)):
        # 세부 평가 결과 초기화
        result = {
            'index': i,
            'db_id': db_id,
            'pred_sql': pred_sql,
            'gold_sql': gold_sql,
            'exact_match': False,
            'execution_match': False
        }

        # DB 경로 구성
        db_path = os.path.join(db_dir, db_id, f"{db_id}.sqlite")
        if not os.path.exists(db_path):
            print(f"경고: 데이터베이스 파일을 찾을 수 없음: {db_path}")
            result['error'] = f"DB 파일을 찾을 수 없음: {db_path}"
            scores['details'].append(result)
            continue

        # 실행 정확도 평가
        try:
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()

            # 예측 SQL 실행
            cursor.execute(pred_sql)
            pred_result = cursor.fetchall()
            result['pred_result_count'] = len(pred_result)

            # 정답 SQL 실행
            cursor.execute(gold_sql)
            gold_result = cursor.fetchall()
            result['gold_result_count'] = len(gold_result)

            # 결과 비교
            execution_match = set(pred_result) == set(gold_result)
            if execution_match:
                scores['execution'] += 1
                result['execution_match'] = True

            conn.close()
        except Exception as e:
            result['error'] = f"실행 오류: {str(e)}"

        # 정확한 일치 평가 (문자열 기반 단순 비교)
        if normalize_sql(pred_sql) == normalize_sql(gold_sql):
            scores['exact_match'] += 1
            result['exact_match'] = True

        # 결과 저장
        scores['details'].append(result)

    # 백분율로 변환
    if scores['total'] > 0:
        scores['exact_match_pct'] = scores['exact_match'] / scores['total'] * 100
        scores['execution_pct'] = scores['execution'] / scores['total'] * 100

    return scores

# 평가 실행 함수
def run_evaluation(model_path=None, sample_size=10, show_all=False):
    """
    모델 평가를 실행합니다.

    Args:
        model_path: 평가할 모델의 경로 (None이면 현재 메모리의 모델 사용)
        sample_size: 평가할 예제 수 (None이면 전체 데이터셋 사용)
        show_all: 모든 예제 결과를 표시할지 여부
    """
    try:
        # 필요한 라이브러리 먼저 임포트
        from unsloth import FastLanguageModel
        import torch
        import pandas as pd
        from IPython.display import display

        # 경로 설정 및 존재 확인
        dev_file = os.path.join(SPIDER_DIR, "dev.json")
        tables_file = os.path.join(SPIDER_DIR, "tables.json")
        db_dir = os.path.join(SPIDER_DIR, "database")

        # 경로 확인
        for path, name in [(dev_file, "dev.json"), (tables_file, "tables.json"), (db_dir, "database")]:
            if not os.path.exists(path):
                print(f"{name} 경로를 찾을 수 없습니다: {path}")
                print(f"SPIDER_DIR = {SPIDER_DIR}")
                print("SPIDER_DIR 변수가 올바르게 설정되었는지 확인하세요.")
                return

        # 개발 데이터셋 로드
        print(f"개발 데이터셋 로드 중: {dev_file}")
        with open(dev_file, 'r') as f:
            dev_data = json.load(f)

        # 샘플 크기가 지정된 경우 예제 수 제한
        if sample_size and sample_size < len(dev_data):
            print(f"전체 {len(dev_data)}개 중 {sample_size}개 예제로 샘플링")
            dev_data = random.sample(dev_data, sample_size)

        # 테이블 스키마 로드
        print(f"테이블 스키마 로드 중: {tables_file}")
        tables_data = load_tables_data(tables_file)
        if not tables_data:
            print("테이블 데이터를 로드하지 못했습니다. 평가를 종료합니다.")
            return

        schema_mapping = create_schema_mapping(tables_data)
        print(f"{len(schema_mapping)}개 데이터베이스에 대한 스키마 매핑 생성")

        # 모델이 메모리에 로드된 상태일 때는 그대로 사용, 아니면 로드
        global model, tokenizer
        if model_path:
            print(f"모델을 {model_path}에서 로드합니다...")
            model, tokenizer = FastLanguageModel.from_pretrained(
                model_path,
                device_map="auto",
                load_in_4bit=True
            )

        # 모델 확인
        if 'model' not in globals() or 'tokenizer' not in globals():
            print("모델과 토크나이저가 로드되지 않았습니다.")
            return

        # 추론 모드로 설정
        if hasattr(model, "for_inference"):
            print("모델을 추론 모드로 설정합니다...")
            FastLanguageModel.for_inference(model)

        # SQL 예측 생성
        preds = []
        golds = []
        questions = []

        print("\nSQL 예측 생성 중:")
        for example in tqdm(dev_data):
            question = example["question"]
            db_id = example["db_id"]
            gold_sql = example["query"]

            # SQL 쿼리 생성
            pred_sql = generate_sql_query(model, tokenizer, schema_mapping, question, db_id)

            # 결과 저장
            questions.append(question)
            preds.append((pred_sql, db_id))
            golds.append((gold_sql, db_id))

        # 평가 수행
        print("\n평가 실행 중...")
        scores = evaluate_predictions(preds, golds, db_dir, tables_file)

        # 결과 출력
        print("\n평가 결과:")
        print(f"총 예제 수: {scores['total']}")
        print(f"완전 일치 정확도: {scores['exact_match_pct']:.2f}% ({scores['exact_match']}/{scores['total']})")
        print(f"실행 정확도: {scores['execution_pct']:.2f}% ({scores['execution']}/{scores['total']})")

        # 결과 데이터프레임 생성
        results = []
        for i, detail in enumerate(scores['details']):
            results.append({
                "예제 번호": i + 1,
                "질문": questions[i],
                "DB": detail['db_id'],
                "정확한 일치": "✅" if detail['exact_match'] else "❌",
                "실행 일치": "✅" if detail['execution_match'] else "❌",
                "예측 SQL": detail['pred_sql'],
                "정답 SQL": detail['gold_sql']
            })

        # 데이터프레임 생성
        df = pd.DataFrame(results)

        # show_all이 True이면 모든 예제 표시, 아니면 일부만 표시
        if show_all:
            print(f"\n모든 예제 결과 ({len(results)}개):")
            display(df)
        else:
            # 최대 5개 샘플만 표시
            max_samples = min(5, len(results))
            print(f"\n샘플 예제 결과 ({max_samples}개):")
            display(df.head(max_samples))
            if len(results) > max_samples:
                print(f"...외 {len(results) - max_samples}개 더 있음. 모두 보려면 show_all=True 옵션 사용")

        return df, scores

    except Exception as e:
        print(f"평가 중 오류 발생: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# 메인 평가 예시 (모든 예제 평가 결과를 보려면 show_all=True 옵션 추가)
# run_evaluation(sample_size=10, show_all=True)  # 10개 예제 전체 결과 표시

In [ ]:
# 모든 예제 예측 결과를 보여주는 함수
def show_all_predictions(model_path=None, sample_size=None):
    """
    모든 예제에 대한 SQL 예측 결과를 생성하여 표시합니다.

    Args:
        model_path: 평가할 모델의 경로 (None이면 현재 메모리의 모델 사용)
        sample_size: 평가할 예제 수 (None이면 전체 데이터셋 사용)
    """
    try:
        # 필요한 라이브러리 먼저 임포트
        from unsloth import FastLanguageModel
        import torch
        import pandas as pd
        from IPython.display import display

        # 경로 설정 및 존재 확인
        dev_file = os.path.join(SPIDER_DIR, "dev.json")
        tables_file = os.path.join(SPIDER_DIR, "tables.json")

        # 경로 확인
        for path, name in [(dev_file, "dev.json"), (tables_file, "tables.json")]:
            if not os.path.exists(path):
                print(f"{name} 경로를 찾을 수 없습니다: {path}")
                print(f"SPIDER_DIR = {SPIDER_DIR}")
                return

        # 개발 데이터셋 로드
        print(f"개발 데이터셋 로드 중: {dev_file}")
        with open(dev_file, 'r') as f:
            dev_data = json.load(f)

        # 샘플 크기가 지정된 경우 예제 수 제한
        if sample_size and sample_size < len(dev_data):
            print(f"전체 {len(dev_data)}개 중 {sample_size}개 예제로 샘플링")
            dev_data = random.sample(dev_data, sample_size)

        # 테이블 스키마 로드
        print(f"테이블 스키마 로드 중: {tables_file}")
        tables_data = load_tables_data(tables_file)
        if not tables_data:
            print("테이블 데이터를 로드하지 못했습니다. 종료합니다.")
            return

        schema_mapping = create_schema_mapping(tables_data)
        print(f"{len(schema_mapping)}개 데이터베이스에 대한 스키마 매핑 생성")

        # 모델이 메모리에 로드된 상태일 때는 그대로 사용, 아니면 로드
        global model, tokenizer
        if model_path:
            print(f"모델을 {model_path}에서 로드합니다...")
            model, tokenizer = FastLanguageModel.from_pretrained(
                model_path,
                device_map="auto",
                load_in_4bit=True
            )

        # 모델 확인
        if 'model' not in globals() or 'tokenizer' not in globals():
            print("모델과 토크나이저가 로드되지 않았습니다.")
            return

        # 추론 모드로 설정
        if hasattr(model, "for_inference"):
            print("모델을 추론 모드로 설정합니다...")
            FastLanguageModel.for_inference(model)

        # SQL 예측 생성
        results = []
        print("\nSQL 예측 생성 중:")
        for i, example in enumerate(tqdm(dev_data)):
            question = example["question"]
            db_id = example["db_id"]
            gold_sql = example["query"]

            # SQL 쿼리 생성
            pred_sql = generate_sql_query(model, tokenizer, schema_mapping, question, db_id)

            # 결과 저장
            results.append({
                "예제 #": i+1,
                "질문": question,
                "DB": db_id,
                "예측 SQL": pred_sql,
                "정답 SQL": gold_sql
            })

        # 데이터프레임으로 변환
        df = pd.DataFrame(results)

        # 결과 표시
        print(f"\n총 {len(results)}개 예제 평가 완료")

        # 데이터프레임 표시
        display(df)

        return df

    except Exception as e:
        print(f"오류 발생: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# 함수 호출 (필요시 주석 해제)
# show_all_predictions(sample_size=10)  # 10개 예제만 표시

In [ ]:
import pandas as pd
import random
import re
import sqlite3

def normalize_sql(sql):
    import re

    if not isinstance(sql, str):
        return sql

    # 1. 따옴표로 묶인 값들을 추출하여 임시 저장
    quoted_values = []

    def replace_quotes(match):
        content = match.group(1) or match.group(2)
        placeholder = f"__QUOTED_VALUE_{len(quoted_values)}__"
        quoted_values.append(content)
        return f"'{placeholder}'"

    pattern = r"'([^']*)'|\"([^\"]*)\""
    sql_with_placeholders = re.sub(pattern, replace_quotes, sql)

    # 2. 나머지 SQL 정규화 (식별자, 키워드 등)
    normalized_sql = sql_with_placeholders.lower().strip()

    # 연속된 공백을 하나로 변환
    normalized_sql = re.sub(r'\s+', ' ', normalized_sql)

    # 쉼표 주변 공백 정규화
    normalized_sql = re.sub(r'\s*,\s*', ', ', normalized_sql)

    # 문장 끝 세미콜론(;) 제거
    normalized_sql = re.sub(r';+\s*$', '', normalized_sql)

    # 연산자 주변 공백 정규화 - 개선된 버전
    # 비교 연산자를 먼저 처리 (순서 중요!)
    comparison_ops = ['!=', '<>', '>=', '<=', '=', '>', '<']
    for op in comparison_ops:
        # 연산자 주변의 모든 공백을 제거한 후 양쪽에 공백 하나씩 추가
        pattern = r'\s*' + re.escape(op) + r'\s*'
        normalized_sql = re.sub(pattern, f' {op} ', normalized_sql)

    # 산술 연산자
    arithmetic_ops = ['+', '-', '*', '/']
    for op in arithmetic_ops:
        pattern = r'\s*' + re.escape(op) + r'\s*'
        normalized_sql = re.sub(pattern, f' {op} ', normalized_sql)

    # 키워드 연산자 (이미 공백이 포함되어 있음)
    keyword_ops = ['and', 'or', 'not', 'in', 'like', 'between']
    for op in keyword_ops:
        # 단어 경계를 사용하여 정확한 매칭
        pattern = r'\s+' + op + r'\s+'
        normalized_sql = re.sub(pattern, f' {op} ', normalized_sql, flags=re.IGNORECASE)

    # 중복 공백 제거
    normalized_sql = re.sub(r'\s+', ' ', normalized_sql)

    # 3. 저장했던 원래 값들 다시 넣기
    for i, value in enumerate(quoted_values):
        placeholder = f"__quoted_value_{i}__"
        normalized_sql = normalized_sql.replace(f"'{placeholder}'", f"'{value}'")

    return normalized_sql.strip()


def execute_sql(db_path, sql):
    """SQLite 데이터베이스에서 SQL을 실행하고 결과를 반환합니다."""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(sql)
        results = cursor.fetchall()
        conn.close()
        return True, results
    except Exception as e:
        return False, str(e)

def compare_sql_results(result1, result2):
    """두 SQL 실행 결과가 동일한지 비교합니다."""
    if result1 is None or result2 is None:
        return False

    # 결과를 정렬된 집합으로 변환하여 비교 (순서 무관)
    try:
        set1 = set(map(tuple, result1))
        set2 = set(map(tuple, result2))
        return set1 == set2
    except:
        # 단순 비교
        return result1 == result2

def generate_sql(model, tokenizer, prompt):
    """프롬프트로부터 SQL 쿼리를 생성합니다."""
    # 채팅 메시지로 포맷팅
    messages = [{"role": "user", "content": prompt}]

    # 채팅 템플릿 적용
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # SQL 쿼리 생성
    outputs = model.generate(
        inputs,
        max_new_tokens=150,
        temperature=0.1,
        do_sample=True,
        use_cache=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # 응답 디코딩
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 프롬프트 이후의 응답만 추출
    if prompt in response:
        sql_part = response.split(prompt)[-1].strip()
    else:
        # 프롬프트를 찾을 수 없는 경우, 'assistant' 태그 후의 내용 추출
        sql_part = response

    # 'assistant' 태그 제거
    if sql_part.startswith("assistant"):
        sql_part = sql_part[len("assistant"):].strip()

    # SQL 쿼리만 추출 (첫 번째 SQL문)
    lines = sql_part.split('\n')
    sql_query = ""
    for line in lines:
        line = line.strip()
        if line.lower().startswith(('select', 'with', '(select')):
            sql_query = line
            break

    # SQL 쿼리를 찾지 못한 경우 전체 응답 반환
    if not sql_query:
        sql_query = sql_part.strip()

    return sql_query

def run_evaluation_ko_from_csv(model, tokenizer, n_samples=10, show_all=False, random_seed=42, check_execution=True):
    """
    CSV 파일(spider_validation_final.csv)에서 한국어 질문을 사용하여 모델을 평가합니다.

    Args:
        model: 평가할 모델
        tokenizer: 토크나이저
        n_samples: 평가할 샘플 수 (기본값: 10)
        show_all: 모든 예시를 보여줄지 여부 (기본값: False)
        random_seed: 랜덤 시드 (기본값: 42)
        check_execution: SQL 실행 결과도 비교할지 여부 (기본값: True)
    """
    try:
        # CSV 파일 경로
        csv_path = os.path.join(SPIDER_DIR, 'spider_validation_final.csv')

        if not os.path.exists(csv_path):
            print(f"❌ CSV 파일을 찾을 수 없습니다: {csv_path}")
            return

        # CSV 파일 로드
        print(f"Loading CSV file from {csv_path}...")
        df = pd.read_csv(csv_path)

        # 데이터 확인
        print(f"Total examples in CSV: {len(df)}")
        print(f"Columns: {df.columns.tolist()}")

        # 필요한 열이 있는지 확인
        required_columns = ['db_id', 'question_ko', 'query']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print(f"❌ Missing required columns: {missing_columns}")
            return

        # 랜덤 샘플 선택
        random.seed(random_seed)
        sample_indices = random.sample(range(len(df)), min(n_samples, len(df)))
        sampled_df = df.iloc[sample_indices]

        print(f"\nEvaluating {len(sampled_df)} random samples...")
        print(f"Execution checking: {'ON' if check_execution else 'OFF'}")
        print("=" * 80)

        # Spider 테이블 정보 로드
        tables_file = os.path.join(SPIDER_DIR, 'tables.json')
        tables_data = load_tables_data(tables_file)
        schema_dict = create_schema_mapping(tables_data)

        # 데이터베이스 디렉토리
        db_dir = os.path.join(SPIDER_DIR, 'database')

        results = []

        for idx, row in sampled_df.iterrows():
            db_id = row['db_id']
            question_ko = row['question_ko']
            gold_sql = row['query']

            # 스키마 정보 가져오기
            if db_id not in schema_dict:
                print(f"❌ Database '{db_id}' not found in schema")
                continue

            schema_info = get_schema_text(db_id, schema_dict)

            # 프롬프트 생성
            prompt = f"""다음 데이터베이스 스키마를 참고하여 질문에 대한 SQL 쿼리를 생성하세요.

### 데이터베이스 스키마:
{schema_info}

### 질문: {question_ko}

### SQL 쿼리:"""

            # 모델 예측
            pred_sql = generate_sql(model, tokenizer, prompt)

            # SQL 정규화하여 비교
            normalized_gold = normalize_sql(gold_sql)
            normalized_generated = normalize_sql(pred_sql)

            # 결과 저장
            result = {
                'index': idx,
                'db_id': db_id,
                'question_ko': question_ko,
                'gold_sql': gold_sql,
                'generated_sql': pred_sql,
                'normalized_gold': normalized_gold,
                'normalized_generated': normalized_generated,
                'exact_match': normalized_gold == normalized_generated,
                'execution_match': False,
                'execution_error': None
            }

            # SQL 실행 결과 비교
            if check_execution:
                db_path = os.path.join(db_dir, db_id, f"{db_id}.sqlite")
                if os.path.exists(db_path):
                    # Gold SQL 실행
                    gold_success, gold_result = execute_sql(db_path, gold_sql)
                    # Generated SQL 실행
                    gen_success, gen_result = execute_sql(db_path, pred_sql)

                    if gold_success and gen_success:
                        result['execution_match'] = compare_sql_results(gold_result, gen_result)
                    else:
                        if not gen_success:
                            result['execution_error'] = gen_result
                else:
                    result['execution_error'] = f"Database file not found: {db_path}"

            results.append(result)

            # 진행 상황 출력
            print(f"\nExample {len(results)}/{len(sampled_df)}:")
            print(f"Database: {db_id}")
            print(f"Question (KO): {question_ko}")
            print(f"Gold SQL: {gold_sql}")
            print(f"Generated SQL: {pred_sql}")
            print(f"Exact Match: {'✅' if result['exact_match'] else '❌'}")

            if check_execution:
                if result['execution_error']:
                    print(f"Execution: ❌ (Error: {result['execution_error'][:100]}...)")
                else:
                    print(f"Execution Match: {'✅' if result['execution_match'] else '❌'}")

            print("-" * 80)

        # 전체 결과 요약
        total_examples = len(results)
        exact_match_count = sum(1 for r in results if r['exact_match'])
        execution_match_count = sum(1 for r in results if r['execution_match'])

        exact_match_accuracy = exact_match_count / total_examples * 100 if total_examples > 0 else 0
        execution_accuracy = execution_match_count / total_examples * 100 if total_examples > 0 else 0

        print(f"\n{'='*80}")
        print(f"Evaluation Summary (Korean Questions from CSV):")
        print(f"Total examples evaluated: {total_examples}")
        print(f"Exact match: {exact_match_count}/{total_examples} ({exact_match_accuracy:.2f}%)")

        if check_execution:
            print(f"Execution match: {execution_match_count}/{total_examples} ({execution_accuracy:.2f}%)")
            execution_errors = sum(1 for r in results if r['execution_error'] is not None)
            print(f"Execution errors: {execution_errors}")

        print(f"{'='*80}")

        # 모든 결과를 DataFrame으로 보여주기
        if show_all or n_samples <= 10:
            print("\n### All Results ###")
            results_df = pd.DataFrame(results)
            results_df['exact_match_icon'] = results_df['exact_match'].apply(lambda x: '✅' if x else '❌')
            results_df['execution_match_icon'] = results_df['execution_match'].apply(lambda x: '✅' if x else '❌')

            display_columns = ['index', 'db_id', 'question_ko', 'exact_match_icon']
            if check_execution:
                display_columns.append('execution_match_icon')

            print(results_df[display_columns].to_string(index=False))

        return results

    except Exception as e:
        print(f"❌ Error during evaluation: {str(e)}")
        import traceback
        traceback.print_exc()

# 사용 예시:
results = run_evaluation_ko_from_csv(model, tokenizer, n_samples=1034, show_all=False, check_execution=True, random_seed=15)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Gold SQL: SELECT COUNT( DISTINCT Continent) FROM country AS T1 JOIN countrylanguage AS T2 ON T1.Code  =  T2.CountryCode WHERE T2.Language  =  "Chinese"
Generated SQL: SELECT count(DISTINCT T1.continent) FROM country AS T1 JOIN countrylanguage AS T2 ON T1.code = T2.countrycode WHERE T2.language = "Chinese"
Exact Match: ❌
Execution Match: ✅
--------------------------------------------------------------------------------

Example 389/1034:
Database: flight_2
Question (KO): APG에서 출발하는 항공편의 항공편 번호를 알려주세요.
Gold SQL: SELECT FlightNo FROM FLIGHTS WHERE SourceAirport  =  "APG"
Generated SQL: SELECT T1.FlightNo FROM Flights AS T1 JOIN Airports AS T2 ON T1.DestAirport = T2.AirportCode WHERE T2.AirportName = "APG"
Exact Match: ❌
Execution Match: ✅
--------------------------------------------------------------------------------

Example 390/1034:
Database: flight_2
Question (KO): 어떤 도시가 가장 많은 도착 항공편을 가지고 있습니까?
Gold SQL: SELECT T1.City FROM AIRPORTS AS T1 JOIN FLI